In [15]:
import sys
sys.path.append('/scratch/cloned_repositories/keras-unet/')

import keras_unet

import glob
import imageio
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [2]:
random_seed = 69

In [3]:
weights_dir = '/scratch/single_cell/EPFL/weights'

In [7]:
image_filenames = glob.glob("/scratch/single_cell/EPFL/raw/*.png")
label_filenames = glob.glob("/scratch/single_cell/EPFL/labels/data*.png")

image_filenames.sort()
label_filenames.sort()

In [8]:
images = []
labels = []

for image_filename, label_filename in zip(image_filenames, label_filenames):
    image = imageio.imread(image_filename)
    images.append(image)
    
    label = imageio.imread(label_filename)
    labels.append(label)

images = np.array(images, dtype=np.float32) / 255.
labels = np.array(labels, dtype=np.float32) / 255.

images = np.expand_dims(images, axis=-1)
labels = np.expand_dims(labels, axis=-1)

print(images.min(), images.max(), "    ", labels.min(), labels.max())
print(images.shape, "    ", labels.shape)

0.007843138 1.0      0.003921569 1.0
(165, 768, 1024, 1)      (165, 768, 1024, 1)


In [11]:
x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.5, random_state=random_seed)
# x_train = images[: 83]
# y_train = labels[: 83]

# x_val = images[83: ]
# y_val = labels[83: ]

In [12]:
print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

x_train:  (82, 768, 1024, 1)
y_train:  (82, 768, 1024, 1)
x_val:  (83, 768, 1024, 1)
y_val:  (83, 768, 1024, 1)


In [16]:
from keras_unet.utils import get_augmented

train_gen = get_augmented(
    x_train, y_train, batch_size=2,
    data_gen_args = dict(
        rotation_range=15.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=50,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant'
    ))

In [17]:
from keras_unet.models import custom_unet

input_shape = x_train[0].shape

model = custom_unet(
    input_shape,
    use_batch_norm=False,
    num_classes=1,
    filters=64,
    dropout=0.2,
    output_activation='sigmoid'
)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 768, 1024, 1 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 768, 1024, 64 640         input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout2d (SpatialDropo (None, 768, 1024, 64 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 768, 1024, 64 36928       spatial_dropout2d[0][0]          
______________________________________________________________________________________________

In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint


model_filename = 'custom_unet_v0.h5'
callback_checkpoint = ModelCheckpoint(
    model_filename, 
    verbose=1, 
    monitor='val_loss', 
    save_best_only=True,
)

In [21]:
from tensorflow.keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

model.compile(
    #optimizer=Adam(), 
    optimizer=SGD(lr=0.01, momentum=0.99),
    loss='binary_crossentropy',
    #loss=jaccard_distance,
    metrics=[iou, iou_thresholded]
)

In [ ]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=100,
    epochs=10,
    
    validation_data=(x_val, y_val),
    callbacks=[callback_checkpoint]
)

  ...
    to  
  ['...']
Train for 100 steps, validate on 83 samples
Epoch 1/10
  4/100 [>.............................] - ETA: 31:27 - loss: 0.4424 - iou: 0.8504 - iou_thresholded: 0.7893